In [2]:
import clip
import numpy as np
import json
import os
import os.path as osp
import torch
import pyvista as pv
import matplotlib.pyplot as plt
from PIL import Image
import trimesh
import time
import optuna
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

## Rendering Setup

In [30]:
# --- Hyperparameter Setup ---

hparams = {
    # Camera
    "camera_distance_factor": 3.11/6.,
    "camera_angle": 45.,
    "focal_length": 6.,
    # Image
    "resolution": 224,
    # Phong
    "ambient_coeff": 0.5,
    "diffuse_coeff": 0.7,
    "specular_coeff": 0.0,
    "shininess": 3.0,
    # Colors
    "object_color": torch.tensor([0.61, 0.61, 0.61]),
    "background_color": torch.tensor([0.46, 0, 0]),
    # Light
    "amb_light_color": torch.tensor([0.15, 0, 0]),
    # light 1
    "light_intensity_1": 1., 
    "light_color": torch.tensor([1.0, 1.0, 1.0]), 
    "light_dir_1": torch.tensor([-1., -1., -1.]), 
    # light 2
    "light_intensity_2": 1.,
    "light_dir_2": torch.tensor([0., 0., -1.]), 
    # light 3
    "light_intensity_3": 0., 
    "light_dir_3": torch.tensor([0., -1., 0.])
}

# --- Rendering Function ---

def phong_model(sdf, points, camera_position, phong_params, light_params, mesh_path, index_tri=None):
    
    # Option 1: Use SDF
    #normals = estimate_normals(sdf, points)
    # Option 2: Use Mesh
    normals = mesh_normals(mesh_path, index_tri)
    view_dirs = points - camera_position 
    light_dir_1 = light_params["light_dir_1"].repeat(points.shape[0], 1) 
    light_dir_2 = light_params["light_dir_2"].repeat(points.shape[0], 1)
    light_dir_3 = light_params["light_dir_3"].repeat(points.shape[0], 1)
    
    # Normalize all vectors
    normals = (normals.T / torch.norm(normals, dim=-1)).T
    light_dir_norm_1 = (light_dir_1.T / torch.norm(light_dir_1, dim=-1)).T
    light_dir_norm_2 = (light_dir_2.T / torch.norm(light_dir_2, dim=-1)).T
    light_dir_norm_3 = (light_dir_3.T / torch.norm(light_dir_3, dim=-1)).T
    view_dir_norm = (view_dirs.T / torch.norm(view_dirs, dim=-1)).T
    
    # Ambient
    ambient = phong_params["ambient_coeff"] * light_params["amb_light_color"] 
    ambient_refl = ambient.repeat(points.shape[0], 1)
    
    # Area light 1
    diffuse_1 = phong_params["diffuse_coeff"] * torch.clamp(torch.sum(-light_dir_norm_1 * normals, dim=-1), min=0.0) * light_params["light_intensity_1"] # [N]
    diffuse_refl_1 = torch.matmul(diffuse_1.unsqueeze(1), light_params["light_color_1"].unsqueeze(0)) # [N, 3]
    reflect_dir_1 = light_dir_norm_1 + (2 * normals.T * torch.clamp(torch.sum(-light_dir_norm_1 * normals, dim=-1), min=0.0)).T
    specular_1 = phong_params["specular_coeff"] * torch.pow(torch.clamp(torch.sum(reflect_dir_1 * -view_dir_norm, dim=-1), min=0.0), phong_params["shininess"]) * light_params["light_intensity_1"] # [N]
    specular_refl_1 = torch.matmul(specular_1.unsqueeze(1), light_params["light_color_1"].unsqueeze(0)) # [N, 3]
    
    # Area light 2
    diffuse_2 = phong_params["diffuse_coeff"] * torch.clamp(torch.sum(-light_dir_norm_2 * normals, dim=-1), min=0.0) * light_params["light_intensity_2"] # [N]
    diffuse_refl_2 = torch.matmul(diffuse_2.unsqueeze(1), light_params["light_color_2"].unsqueeze(0)) # [N, 3]
    reflect_dir_2 = light_dir_norm_2 + (2 * normals.T * torch.clamp(torch.sum(-light_dir_norm_2 * normals, dim=-1), min=0.0)).T
    specular_2 = phong_params["specular_coeff"] * torch.pow(torch.clamp(torch.sum(reflect_dir_2 * -view_dir_norm, dim=-1), min=0.0), phong_params["shininess"]) * light_params["light_intensity_2"] # [N]
    specular_refl_2 = torch.matmul(specular_2.unsqueeze(1), light_params["light_color_2"].unsqueeze(0)) # [N, 3]
    
    # Area light 3
    diffuse_3 = phong_params["diffuse_coeff"] * torch.clamp(torch.sum(-light_dir_norm_3 * normals, dim=-1), min=0.0) * light_params["light_intensity_3"] # [N]
    diffuse_refl_3 = torch.matmul(diffuse_3.unsqueeze(1), light_params["light_color_3"].unsqueeze(0)) # [N, 3]
    reflect_dir_3 = light_dir_norm_3 + (2 * normals.T * torch.clamp(torch.sum(-light_dir_norm_3 * normals, dim=-1), min=0.0)).T
    specular_3 = phong_params["specular_coeff"] * torch.pow(torch.clamp(torch.sum(reflect_dir_3 * -view_dir_norm, dim=-1), min=0.0), phong_params["shininess"]) * light_params["light_intensity_3"] # [N]
    specular_refl_3 = torch.matmul(specular_3.unsqueeze(1), light_params["light_color_3"].unsqueeze(0)) # [N, 3]
    
    # point light
    #diffuse_1 = diffuse_coeff * torch.clamp(torch.sum(-light_dir_norm_1 * normals, dim=-1), min=0.0) * light_intensity_1 # [N]
    #diffuse_refl_1 = torch.matmul(diffuse_1.unsqueeze(1), light_color.unsqueeze(0)) # [N, 3]
    #reflect_dir_1 = light_dir_norm_1 + (2 * normals.T * torch.clamp(torch.sum(-light_dir_norm_1 * normals, dim=-1), min=0.0)).T
    #specular_1 = specular_coeff * torch.pow(torch.clamp(torch.sum(reflect_dir_1 * -view_dir_norm, dim=-1), min=0.0), shininess) * light_intensity_1 # [N]
    #specular_refl_1 = torch.matmul(specular_1.unsqueeze(1), light_color.unsqueeze(0)) # [N, 3]

    return ambient_refl + diffuse_refl_1 + specular_refl_1 + diffuse_refl_2 + specular_refl_2 + diffuse_refl_3 + specular_refl_3

def estimate_normals(sdf, points, epsilon=1e-3):
    sdf_inputs = torch.concat([points,
                              points + torch.tensor([epsilon, 0, 0]),
                              points + torch.tensor([0, epsilon, 0]),
                              points + torch.tensor([0, 0, epsilon])])

    sdf_values = sdf(sdf_inputs).reshape(4, -1)

    # Calculate the gradient using finite differences
    gradient = sdf_values[1:] - sdf_values[0]

    # Normalize the gradient to obtain the estimated normal
    normal = gradient / torch.norm(gradient, p=2, dim=0)

    return normal.T

def sphere_trace(sdf, camera_position, norm_directions, max_length):
    N = norm_directions.shape[0]
    positions = camera_position.unsqueeze(dim=0).repeat(N, 1) # [N, 3]
    total_distances = torch.zeros(N)
    last_distances = torch.ones(N)

    for _ in range(20):
        #mask = torch.logical_and(total_distances < max_length, last_distances > 1e-3)
        not_reached_max_distance = total_distances < max_length
        not_hit_target = torch.abs(last_distances) > 1e-3
        mask = torch.logical_and(not_reached_max_distance, not_hit_target)
        if torch.all(torch.logical_not(mask)):
            break
        distances = sdf(positions[mask])
        steps = (norm_directions[mask].T * distances).T
        positions[mask] += steps
        total_distances[mask] += distances
        last_distances[mask] = distances

    #positions[total_distances > max_length] *= torch.nan
    return positions, total_distances < max_length

def mesh_trace(mesh_path, ray_starts, ray_directions):

    mesh = trimesh.load_mesh(mesh_path)
    
    ray_origins = ray_starts.repeat(ray_directions.shape[0], 1)
    
    intersections, index_ray, index_tri = mesh.ray.intersects_location(ray_origins, ray_directions, multiple_hits=False) 
    
    mask = torch.zeros(ray_directions.shape[0], dtype=torch.bool)
    mask[index_ray] = True
    
    points = torch.FloatTensor(intersections).to(dtype=torch.float32)

    return points, mask, index_tri


def mesh_normals(mesh_path, index_tri):
    mesh = trimesh.load_mesh(mesh_path)
    all_normals = mesh.face_normals
    normals = all_normals[index_tri]
    normals_torch = torch.from_numpy(normals).to(dtype=torch.float32)
    
    return normals_torch

def render(sdf, camera_params, phong_params, light_params, mesh_path=None):
    
    pu = camera_params["resolution_x"]
    pv = camera_params["resolution_y"]
    image = phong_params["background_color"].repeat(pu * pv, 1)
    
    angle_radians = torch.deg2rad_(torch.tensor(camera_params["camera_angle"]))
    camera = torch.tensor([torch.sin(angle_radians), 0, torch.cos(angle_radians)])
    camera_position = camera * (camera_params["camera_distance"] + camera_params["focal_length"]) / camera.norm()
    
    # Normalize the xy value of the current pixel [-0.5, 0.5]
    u_norms = ((torch.arange(pu) + 0.5) / pu - 0.5) * pu/pv
    v_norms = 0.5 - (torch.arange(pv) + 0.5) / pv

    # Calculate the ray directions for all pixels
    directions_unn = torch.cat(torch.meshgrid(u_norms, v_norms, torch.tensor(-camera_params["focal_length"])), dim=-1) 
    directions_unn = directions_unn.reshape((pu*pv, 3)) # [pu, pv, 3] --> [pu*pv, 3] (u1, v1, f)(u1, v2, f)...(u2, v1, f)...

    # rotate about y-axis
    rotation_matrix = torch.tensor([[torch.cos(angle_radians), 0, torch.sin(angle_radians)],
                                    [0, 1, 0],
                                    [-torch.sin(angle_radians), 0, torch.cos(angle_radians)]])
    rotated_directions = torch.matmul(directions_unn, rotation_matrix.T)
    
    transposed_directions = rotated_directions.T #transpose is necessary for normalization
    directions = (transposed_directions / transposed_directions.norm(dim=0)).T # [pu*pv, 3]

    # Option 1: Use SDF
    #hit_positions, hit_mask = sphere_trace(sdf, camera_position, directions, max_ray_length)
    # Option 2: Use Mesh
    intersections, hit_mask, index_tri = mesh_trace(mesh_path, camera_position, directions) 
    
    # Option 1: Use SDF
    #reflections = phong_model(sdf, hit_positions[hit_mask], camera_position, ambient_coeff, diffuse_coeff, specular_coeff, shininess, mesh_path, amb_light_color, light_intensity_1, light_color_1, light_dir_1, light_intensity_2, light_color_2, light_dir_2, light_intensity_3, light_color_3, light_dir_3, index_tri=None)
    # Option 2: Use Mesh
    reflections = phong_model(sdf, intersections, camera_position, phong_params, light_params, mesh_path, index_tri) # mesh alternative

    # Assign a color for objects
    image[hit_mask] = torch.mul(reflections, phong_params["object_color"].repeat(reflections.shape[0], 1))
    image = torch.clamp(image, max=1.0)
    image = image.reshape(pu, pv, 3).transpose(0, 1)

    return image

## Prepare Score Calculation Data

In [31]:
# --- Get Data Annotations ---

path_to_head_annotations = '/Users/katharinaschmid/Text2Head/rendering_data/head_annotations.json'
with open(path_to_head_annotations) as f:
    data = json.load(f)

# --- Prepare Indices ---

# Identity: Gender
male_indices = [index for index, head in enumerate(data['heads']) if head['gender'] == 'man']
female_indices = [index for index, head in enumerate(data['heads']) if head['gender'] == 'woman']

# Identity: Age
young_indices = [index for index, head in enumerate(data['heads']) if head['age'] == 'young']
old_indices = [index for index, head in enumerate(data['heads']) if head['age'] == 'old']

# Identity: Ethnicity
caucasian_indices = [index for index, head in enumerate(data['heads']) if head['ethnicity'] == 'Caucasian'] 
asian_indices = [index for index, head in enumerate(data['heads']) if head['ethnicity'] == 'Asian'] 
african_indices = [index for index, head in enumerate(data['heads']) if head['ethnicity'] == 'African'] 

# Identity: Hairstyle
hat_indices = [index for index, head in enumerate(data['heads']) if head['hairstyle'] == 'hat']
ponytail_indices = [index for index, head in enumerate(data['heads']) if head['hairstyle'] == 'ponytail']
straight_hair_indices = [index for index, head in enumerate(data['heads']) if head['hairstyle'] == 'straight hair']
curly_hair_indices = [index for index, head in enumerate(data['heads']) if head['hairstyle'] == 'curly hair']

# Identity: Beard
beard_indices = [index for index, head in enumerate(data['heads']) if head['beard'] == 'yes']
no_beard_indices = [index for index, head in enumerate(data['heads']) if head['beard'] == 'no']

# Expression: Facial Expression
open_mouth_indices = [index for index, head in enumerate(data['heads']) if head['facial_expression'] == 'open mouth']
smiling_indices = [index for index, head in enumerate(data['heads']) if head['facial_expression'] == 'smiling']
closed_eyes_indices = [index for index, head in enumerate(data['heads']) if head['facial_expression'] == 'closed eyes']
raised_brows_indices = [index for index, head in enumerate(data['heads']) if head['facial_expression'] == 'raised brows']
puffed_cheeks_indices = [index for index, head in enumerate(data['heads']) if head['facial_expression'] == 'puffed_cheeks']

# Expression: Emotion 
sad_indices = [index for index, head in enumerate(data['heads']) if head['emotion'] == 'sad']
happy_indices = [index for index, head in enumerate(data['heads']) if head['emotion'] == 'happy']

# --- Precompute CLIP Text Embeddings ---
captions = []
captions.append('A man') # 0
captions.append('A woman') # 1
captions.append('A young person') # 2
captions.append('An old person') # 3
captions.append('A Caucasian person') # 4
captions.append('An Asian person') # 5
captions.append('An African person') # 6
captions.append('A person wearing a hat') # 7
captions.append('A person with a ponytail') # 8
captions.append('A person with straight hair') # 9
captions.append('A person with curly hair') # 10
captions.append('A person with beard') # 11
captions.append('A person with open mouth') # 12
captions.append('A smiling person') # 13
captions.append('A person with closed eyes') # 14
captions.append('A person with raised brows') # 15
captions.append('A person with puffed cheeks') # 16
captions.append('A sad person') # 17
captions.append('A happy person') # 18

preprocessed_text = clip.tokenize(captions).to(device) # [num_captions, 77]

with torch.no_grad():
    text_features = model.encode_text(preprocessed_text) # [num_captions, 512]
    text_features /= text_features.norm(dim=-1, keepdim=True) # [num_captions, 512]

## Hyperparameter Search for Renderer

In [37]:
# --- Penalty Function to penalize negative Signal Scores ---
def penalty(vector, k):
    vector[vector < 0] *= k
    return vector

def analyze_rendering(hparams):
    
    # --- Get Hyperparameters ---
    camera_params = {
        "camera_distance": hparams['camera_distance_factor'] * hparams['focal_length'], # ensures appropriate head size
        "camera_angle": hparams['camera_angle'],
        "focal_length": hparams['focal_length'],
        "max_ray_length": (hparams['camera_distance_factor'] + 1) * hparams['focal_length'] + 1.5,
        # Image
        "resolution_y": hparams['resolution'],
        "resolution_x": hparams['resolution']
    }
    phong_params = {
        "ambient_coeff": hparams['ambient_coeff'],
        "diffuse_coeff": hparams['diffuse_coeff'],
        "specular_coeff": hparams['specular_coeff'],
        "shininess": hparams['shininess'],
        # Colors
        "object_color": hparams['object_color'],
        "background_color": hparams['background_color']
    }
    
    light_params = {
        "amb_light_color": hparams['amb_light_color'],
        # light 1
        "light_intensity_1": hparams['light_intensity_1'],
        "light_color_1": hparams['light_color'],
        "light_dir_1": hparams['light_dir_1'],
        # light 2
        "light_intensity_2": hparams['light_intensity_2'],
        "light_color_2": hparams['light_color'],
        "light_dir_2": hparams['light_dir_2'],
        # light 3
        "light_intensity_3": hparams['light_intensity_3'],
        "light_color_3": hparams['light_color'],
        "light_dir_3": hparams['light_dir_3'],
    }
    
    # --- Render and Embed Images ---
    folder_path = '/Users/katharinaschmid/Text2Head/rendering_data/meshes/'
    file_list = sorted(os.listdir(folder_path))
    file_list = [file for file in file_list if not file.startswith('.DS_Store')]
    
    image_embeddings_list = []
    with torch.no_grad():
        for mesh in file_list:
            mesh_path = folder_path + mesh
            sdf = None
            rendered_image = render(sdf, camera_params, phong_params, light_params, mesh_path)
            image = (rendered_image * 255).byte()  # Convert to a byte tensor (0-255)
            image_np = image.cpu().numpy()
            image_pil = Image.fromarray(image_np)
            preprocessed_image = preprocess(image_pil).unsqueeze(0) # [1, 3, 224, 224]
            image_features = model.encode_image(preprocessed_image) # [1, 512]
            image_embeddings_list.append(image_features)
        image_embeddings = torch.cat(image_embeddings_list, dim=0)
        image_embeddings /= image_embeddings.norm(dim=-1, keepdim=True)
        
        # --- Get CLIP Similarity and Scores ---
        cosine_similarity = torch.matmul(text_features, image_embeddings.T) #[num_captions*tests, num_images]
        
        # Identity Scores
        i_man_score = torch.mean(cosine_similarity[0, male_indices], dim=-1) - torch.mean(cosine_similarity[0, female_indices], dim=-1)
        i_woman_score = torch.mean(cosine_similarity[1, female_indices], dim=-1) - torch.mean(cosine_similarity[1, male_indices], dim=-1)
        i_young_score = torch.mean(cosine_similarity[2, young_indices], dim=-1) - torch.mean(cosine_similarity[2, old_indices], dim=-1)
        i_old_score = torch.mean(cosine_similarity[3, old_indices], dim=-1) - torch.mean(cosine_similarity[3, young_indices], dim=-1)
        i_caucasian_score = torch.mean(cosine_similarity[4, caucasian_indices], dim=-1) - torch.mean(cosine_similarity[4, (asian_indices + african_indices)], dim=-1)
        i_asian_score = torch.mean(cosine_similarity[5, asian_indices], dim=-1) - torch.mean(cosine_similarity[5, (caucasian_indices + african_indices)], dim=-1)
        # 6 skip african
        i_hat_score = torch.mean(cosine_similarity[7, hat_indices], dim=-1) - torch.mean(cosine_similarity[7, (ponytail_indices + straight_hair_indices + curly_hair_indices)], dim=-1)
        i_ponytail_score = torch.mean(cosine_similarity[8, ponytail_indices], dim=-1) - torch.mean(cosine_similarity[8, (hat_indices + straight_hair_indices + curly_hair_indices)], dim=-1)
        i_straight_hair_score = torch.mean(cosine_similarity[9, straight_hair_indices], dim=-1) - torch.mean(cosine_similarity[9, curly_hair_indices], dim=-1) # ONLY curly hair as opposite!
        i_curly_hair_score = torch.mean(cosine_similarity[10, curly_hair_indices], dim=-1) - torch.mean(cosine_similarity[10, straight_hair_indices], dim=-1) # ONLY straight hair as opposite!
        i_beard_score = torch.mean(cosine_similarity[11, beard_indices], dim=-1) - torch.mean(cosine_similarity[11, no_beard_indices], dim=-1)
        
        # Expression Scores
        e_open_mouth_score = torch.mean(cosine_similarity[12, open_mouth_indices], dim=-1) - torch.mean(cosine_similarity[12, (closed_eyes_indices, raised_brows_indices, puffed_cheeks_indices)], dim=-1) # no smiling as opposite b/c mouth can be open when grinning
        e_smiling_score = torch.mean(cosine_similarity[13, smiling_indices], dim=-1) - torch.mean(cosine_similarity[13, (open_mouth_indices, closed_eyes_indices, raised_brows_indices, puffed_cheeks_indices)], dim=-1)
        e_closed_eyes_score = torch.mean(cosine_similarity[14, closed_eyes_indices], dim=-1) - torch.mean(cosine_similarity[14, (open_mouth_indices, smiling_indices, raised_brows_indices, puffed_cheeks_indices)], dim=-1)
        e_raised_brows_score = torch.mean(cosine_similarity[15, raised_brows_indices], dim=-1) - torch.mean(cosine_similarity[15, (open_mouth_indices, smiling_indices, closed_eyes_indices, puffed_cheeks_indices)], dim=-1)
        e_puffed_cheeks_score = torch.mean(cosine_similarity[16, puffed_cheeks_indices], dim=-1) - torch.mean(cosine_similarity[16, (open_mouth_indices, smiling_indices, closed_eyes_indices, raised_brows_indices)], dim=-1)
        e_sad_score = torch.mean(cosine_similarity[17, sad_indices], dim=-1) - torch.mean(cosine_similarity[17, happy_indices], dim=-1)
        e_happy_score = torch.mean(cosine_similarity[18, happy_indices], dim=-1) - torch.mean(cosine_similarity[18, sad_indices], dim=-1)
        
        # Stack Scores, add penalty, compute average
        i_scores = torch.stack((i_man_score, i_woman_score, i_young_score, i_old_score, i_caucasian_score, i_asian_score, i_hat_score, i_ponytail_score, i_straight_hair_score, i_curly_hair_score, i_beard_score), dim=0)
        e_scores = torch.stack((e_open_mouth_score, e_smiling_score, e_closed_eyes_score, e_raised_brows_score, e_puffed_cheeks_score, e_sad_score, e_happy_score), dim=0)
        print('i_scores: ', i_scores, 'e_scores: ', e_scores)
        scores = torch.stack((i_scores, e_scores), dim=0)
        scores_penalized = penalty(scores, 10)
        scores_avg = torch.mean(scores_penalized, dim=0)
    
    return scores_avg

def objective(trial):
    # Define the hyperparameters to tune and their search spaces
    search_space = {
        # Camera
        "camera_distance_factor": trial.suggest_float('camera_distance_factor', 0.1, 0.8), 
        "camera_angle": trial.suggest_categorical('camera_angle', [0., 15., 30., 45., 55.]), 
        "focal_length": trial.suggest_float('focal_length', 2.0, 9.0),
        # Image
        "resolution": trial.suggest_categorical('resolution', [150, 200, 224, 300]), 
        # Phong
        "ambient_coeff": trial.suggest_float('ambient_coeff', 0.0, 1.0), 
        "diffuse_coeff": trial.suggest_float('diffuse_coeff', 0.0, 1.0), 
        "specular_coeff": trial.suggest_float('specular_coeff', 0.0, 1.0), 
        "shininess": trial.suggest_categorical('shininess', [0.1, 0.5, 1.0, 3.0, 10.]),
        # Colors
        "object_color": torch.tensor([
            trial.suggest_float('object_color_0', 0.0, 1.0),
            trial.suggest_float('object_color_1', 0.0, 1.0),
            trial.suggest_float('object_color_2', 0.0, 1.0)
        ]),
        "background_color": torch.tensor([
            trial.suggest_float('background_color_0', 0.0, 1.0),
            trial.suggest_float('background_color_1', 0.0, 1.0),
            trial.suggest_float('background_color_2', 0.0, 1.0)
        ]),
        # Light
        "amb_light_color": torch.tensor([
            trial.suggest_float('amb_light_color_0', 0.0, 1.0),
            trial.suggest_float('amb_light_color_1', 0.0, 1.0),
            trial.suggest_float('amg_light_color_2', 0.0, 1.0)
        ]),
        # light 1
        "light_intensity_1": trial.suggest_float('light_intensity_1', 0.5, 1.5), 
        "light_color": torch.tensor([
            trial.suggest_float('light_color_0', 0.7, 1.0),
            trial.suggest_float('light_color_1', 0.7, 1.0),
            trial.suggest_float('light_color_2', 0.7, 1.0)
        ]),
        "light_dir_1": torch.tensor([
            trial.suggest_float('light_dir_1_0', -1.0, 0.0),
            trial.suggest_float('light_dir_1_1', -1.0, 0.0),
            trial.suggest_float('light_dir_1_2', -1.0, 0.0)
        ]),
        # light 2
        "light_intensity_2": trial.suggest_float('light_intensity_2', 0.0, 1.), 
        "light_dir_2": torch.tensor([
            0.0,
            trial.suggest_float('light_dir_2_1', -1.0, 0.0),
            trial.suggest_float('light_dir_2_2', -1.0, 0.0)
        ]),
        # light 3
        "light_intensity_3": trial.suggest_float('light_intensity_3', 0.0, 0.7), 
        "light_dir_3": torch.tensor([
            trial.suggest_float('light_dir_2_0', -1.0, 0.5),
            trial.suggest_float('light_dir_2_1', -1.0, 0.0),
            0.0
        ]),
    }
    
    hparams.update(search_space)
    
    return analyze_rendering(hparams)

In [40]:
study = optuna.create_study(storage="sqlite:///optuna_study.db", study_name="test_2", direction='maximize')
study.optimize(objective, n_trials=10)

study.best_params

[I 2023-11-23 15:02:10,052] A new study created in RDB with name: test_2


file_list:  ['01.ply', '02.ply', '03.ply']


[I 2023-11-23 15:02:28,401] Trial 0 finished with value: 0.026060357689857483 and parameters: {'camera_distance_factor': 0.4262159524767436, 'camera_angle': 45.0, 'focal_length': 2.8588476021095603, 'resolution': 224, 'ambient_coeff': 0.37326565625223007, 'diffuse_coeff': 0.38317351714198056, 'specular_coeff': 0.9486262710673882, 'shininess': 0.1, 'object_color_0': 0.42585222969783176, 'object_color_1': 0.9343647618107314, 'object_color_2': 0.40583866285955716, 'background_color_0': 0.6825755786499671, 'background_color_1': 0.12299008231283204, 'background_color_2': 0.5865344696862822, 'amb_light_color_0': 0.40382610739760083, 'amb_light_color_1': 0.5327496805841134, 'amg_light_color_2': 0.5502157681648859, 'light_intensity_1': 0.6163791478537356, 'light_color_0': 0.7305198710849781, 'light_color_1': 0.8587854743569627, 'light_color_2': 0.7135445661080153, 'light_dir_1_0': -0.3947715529507573, 'light_dir_1_1': -0.8557223172563867, 'light_dir_1_2': -0.9508653955941185, 'light_intensity_

file_list:  ['01.ply', '02.ply', '03.ply']


[I 2023-11-23 15:02:41,808] Trial 1 finished with value: 0.024325087666511536 and parameters: {'camera_distance_factor': 0.40916219686271427, 'camera_angle': 55.0, 'focal_length': 2.8636172099657142, 'resolution': 200, 'ambient_coeff': 0.6754888567579762, 'diffuse_coeff': 0.6458600858367192, 'specular_coeff': 0.9827301028175933, 'shininess': 0.5, 'object_color_0': 0.06067822055443284, 'object_color_1': 0.6135365973137048, 'object_color_2': 0.33690484274308075, 'background_color_0': 0.3243916221990384, 'background_color_1': 0.2611450172828872, 'background_color_2': 0.3826185806237734, 'amb_light_color_0': 0.6336746585485075, 'amb_light_color_1': 0.8066451070752583, 'amg_light_color_2': 0.7765048322850088, 'light_intensity_1': 1.2118817755070204, 'light_color_0': 0.707126379242294, 'light_color_1': 0.8698593393309547, 'light_color_2': 0.9275893612939745, 'light_dir_1_0': -0.850562305767666, 'light_dir_1_1': -0.35364776575497603, 'light_dir_1_2': -0.1557939448504324, 'light_intensity_2': 

...

file_list:  ['01.ply', '02.ply', '03.ply']


[I 2023-11-23 15:04:17,222] Trial 8 finished with value: 0.01657561957836151 and parameters: {'camera_distance_factor': 0.4044343656072694, 'camera_angle': 45.0, 'focal_length': 4.0370660899713275, 'resolution': 150, 'ambient_coeff': 0.6077164985849127, 'diffuse_coeff': 0.06201789445225647, 'specular_coeff': 0.6629865885635235, 'shininess': 3.0, 'object_color_0': 0.13636314102303415, 'object_color_1': 0.7775495682271227, 'object_color_2': 0.38929077708301185, 'background_color_0': 0.4301848234777642, 'background_color_1': 0.34892478651347003, 'background_color_2': 0.9376812791242909, 'amb_light_color_0': 0.035654711432842046, 'amb_light_color_1': 0.002191032183236352, 'amg_light_color_2': 0.135841977726189, 'light_intensity_1': 1.4279032914880505, 'light_color_0': 0.8214844133102721, 'light_color_1': 0.824180171801002, 'light_color_2': 0.8599711074806171, 'light_dir_1_0': -0.33659307857528675, 'light_dir_1_1': -0.06324836874070261, 'light_dir_1_2': -0.7694446415463743, 'light_intensity

file_list:  ['01.ply', '02.ply', '03.ply']


[I 2023-11-23 15:04:29,220] Trial 9 finished with value: 0.02276451140642166 and parameters: {'camera_distance_factor': 0.6753698837472806, 'camera_angle': 30.0, 'focal_length': 8.756486743111676, 'resolution': 300, 'ambient_coeff': 0.06377967326707223, 'diffuse_coeff': 0.5430795717519239, 'specular_coeff': 0.7666935323799584, 'shininess': 0.5, 'object_color_0': 0.15414281757429626, 'object_color_1': 0.04261708579975654, 'object_color_2': 0.40158150794307534, 'background_color_0': 0.2380612637955294, 'background_color_1': 0.6809476463705121, 'background_color_2': 0.9612775422289118, 'amb_light_color_0': 0.3429750517171205, 'amb_light_color_1': 0.3413465259261913, 'amg_light_color_2': 0.11117924229735066, 'light_intensity_1': 1.3043296907222266, 'light_color_0': 0.7296652263396699, 'light_color_1': 0.7798716958394977, 'light_color_2': 0.8535753814128466, 'light_dir_1_0': -0.9672728878682709, 'light_dir_1_1': -0.4845872577606737, 'light_dir_1_2': -0.22091586222694437, 'light_intensity_2'

{'camera_distance_factor': 0.4262159524767436,
 'camera_angle': 45.0,
 'focal_length': 2.8588476021095603,
 'resolution': 224,
 'ambient_coeff': 0.37326565625223007,
 'diffuse_coeff': 0.38317351714198056,
 'specular_coeff': 0.9486262710673882,
 'shininess': 0.1,
 'object_color_0': 0.42585222969783176,
 'object_color_1': 0.9343647618107314,
 'object_color_2': 0.40583866285955716,
 'background_color_0': 0.6825755786499671,
 'background_color_1': 0.12299008231283204,
 'background_color_2': 0.5865344696862822,
 'amb_light_color_0': 0.40382610739760083,
 'amb_light_color_1': 0.5327496805841134,
 'amg_light_color_2': 0.5502157681648859,
 'light_intensity_1': 0.6163791478537356,
 'light_color_0': 0.7305198710849781,
 'light_color_1': 0.8587854743569627,
 'light_color_2': 0.7135445661080153,
 'light_dir_1_0': -0.3947715529507573,
 'light_dir_1_1': -0.8557223172563867,
 'light_dir_1_2': -0.9508653955941185,
 'light_intensity_2': 0.5728753113510167,
 'light_dir_2_1': -0.7932316202321615,
 'light